Imports

In [1]:
!pip install transformers
!pip install datasets


[notice] A new release of pip available: 22.3.1 -> 23.0
[notice] To update, run: C:\Users\elena\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


     -------------------------------------- 462.8/462.8 kB 4.9 MB/s eta 0:00:00
     -------------------------------------- 110.5/110.5 kB 6.3 MB/s eta 0:00:00
     --------------------------------------- 20.6/20.6 MB 12.8 MB/s eta 0:00:00
     -------------------------------------- 319.7/319.7 kB 9.7 MB/s eta 0:00:00
     -------------------------------------- 134.3/134.3 kB 8.3 MB/s eta 0:00:00
     -------------------------------------- 143.0/143.0 kB 4.3 MB/s eta 0:00:00
     ---------------------------------------- 56.1/56.1 kB 3.1 MB/s eta 0:00:00
  Attempting uninstall: charset-normalizer
    Found existing installation: charset-normalizer 3.0.1
    Uninstalling charset-normalizer-3.0.1:
      Successfully uninstalled charset-normalizer-3.0.1


  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.

[notice] A new release of pip available: 22.3.1 -> 23.0
[notice] To update, run: C:\Users\elena\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [2]:
import numpy as np
import pandas as pd
from sklearn import metrics 
import transformers
import torch
from transformers import AutoModel, AutoTokenizer
from transformers import AutoModelForSequenceClassification
from transformers import TrainingArguments, Trainer
from datasets import load_dataset
from sklearn.metrics import f1_score, roc_auc_score, accuracy_score
from transformers import EvalPrediction


Lectura de los csv como Datasets

In [4]:
data_files = {"train": "./train.csv", "validation": "./validation.csv"}
dataset = load_dataset("csv", data_files=data_files) 

Using custom data configuration default-61d07a8921541ff0


Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

C:\Users\elena\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\datasets\download\streaming_download_manager.py:776: FutureWarning: the 'mangle_dupe_cols' keyword is deprecated and will be removed in a future version. Please take steps to stop the use of 'mangle_dupe_cols'
  return pd.read_csv(xopen(filepath_or_buffer, "rb", use_auth_token=use_auth_token), **kwargs)


Generating validation split: 0 examples [00:00, ? examples/s]

C:\Users\elena\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\datasets\download\streaming_download_manager.py:776: FutureWarning: the 'mangle_dupe_cols' keyword is deprecated and will be removed in a future version. Please take steps to stop the use of 'mangle_dupe_cols'
  return pd.read_csv(xopen(filepath_or_buffer, "rb", use_auth_token=use_auth_token), **kwargs)


Dataset csv downloaded and prepared to C:/Users/elena/.cache/huggingface/datasets/csv/default-61d07a8921541ff0/0.0.0/6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

In [5]:
dataset

DatasetDict({
    train: Dataset({
        features: ['text', 'service', 'metric', 'objetive', 'remedies', 'claim', 'exception', 'definition'],
        num_rows: 58
    })
    validation: Dataset({
        features: ['text', 'service', 'metric', 'objetive', 'remedies', 'claim', 'exception', 'definition'],
        num_rows: 30
    })
})

In [6]:
example = dataset['train'][:10]
example

{'text': ['Last Updated: May 25, 2022 This Amazon Compute Service Level Agreement (this “SLA”) is a policy governing the use of Amazon Elastic Compute Cloud (“Amazon EC2”)* and applies separately to each account using Amazon EC2.',
  'In the event of a conflict between the terms of this SLA and the terms of the AWS Customer Agreement or other agreement with us governing your use of our Services (the “Agreement”), the terms and conditions of this SLA apply, but only to the extent of such conflict.',
  '*For purposes of this SLA, Amazon EC2 includes any Amazon Elastic Graphics, Amazon Elastic Inference, and Elastic IP Address resources purchased with the relevant Amazon EC2 instance(s).',
  'SLAs AWS makes two SLA commitments for Amazon EC2: (1) a Region-Level SLA that governs Amazon EC2 deployed across multiple AZs or regions, and (2) an Instance-Level SLA that governs Amazon EC2 instances individually.',
  'Region-Level SLA For Amazon EC2 with all running instances deployed concurrentl

In [7]:
labels = [label for label in dataset['train'].features.keys() if label not in ['text']]
id2label = {idx:label for idx, label in enumerate(labels)}
label2id = {label:idx for idx, label in enumerate(labels)}
labels

['service',
 'metric',
 'objetive',
 'remedies',
 'claim',
 'exception',
 'definition']

Preprocess data

In [8]:
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

def preprocess_data(examples):
  # take a batch of texts
  text = examples["text"]
  # encode them
  encoding = tokenizer(text, padding="max_length", truncation=True, max_length=128)
  # add labels
  labels_batch = {k: examples[k] for k in examples.keys() if k in labels}
  # create numpy array of shape (batch_size, num_labels)
  labels_matrix = np.zeros((len(text), len(labels)))
  # fill numpy array
  for idx, label in enumerate(labels):
    labels_matrix[:, idx] = labels_batch[label]

  encoding["labels"] = labels_matrix.tolist()
  
  return encoding

C:\Users\elena\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\huggingface_hub\file_download.py:129: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\elena\.cache\huggingface\hub. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


In [9]:
encoded_dataset = dataset.map(preprocess_data, batched=True, remove_columns=dataset['train'].column_names)
encoded_dataset

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 58
    })
    validation: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 30
    })
})

In [10]:
example = encoded_dataset['train'][0]
print(example)

{'input_ids': [101, 2197, 7172, 1024, 2089, 2423, 1010, 16798, 2475, 2023, 9733, 24134, 2326, 2504, 3820, 1006, 2023, 1523, 22889, 2050, 1524, 1007, 2003, 1037, 3343, 8677, 1996, 2224, 1997, 9733, 21274, 24134, 6112, 1006, 1523, 9733, 14925, 2475, 1524, 1007, 1008, 1998, 12033, 10329, 2000, 2169, 4070, 2478, 9733, 14925, 2475, 1012, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1,

In [11]:
tokenizer.decode(example['input_ids'])

'[CLS] last updated : may 25, 2022 this amazon compute service level agreement ( this “ sla ” ) is a policy governing the use of amazon elastic compute cloud ( “ amazon ec2 ” ) * and applies separately to each account using amazon ec2. [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD]'

In [12]:
example['labels']

[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]

In [13]:
[id2label[idx] for idx, label in enumerate(example['labels']) if label == 1.0]

['service']

In [14]:
encoded_dataset.set_format("torch")

Define model Bert 

Usaremos el preentrenado con la configuración por defecto

In [15]:
model = AutoModelForSequenceClassification.from_pretrained("bert-base-uncased", 
                                                           problem_type="multi_label_classification", 
                                                           num_labels=len(labels),
                                                           id2label=id2label,
                                                           label2id=label2id)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

Train the model!!

Usaremos el método de entrenamiento de la API de Hugging face. Requiere dos cosas.

In [16]:
batch_size = 8
metric_name = "f1"

In [17]:
args = TrainingArguments(
    f"bert-finetuned-sem_eval-english",
    evaluation_strategy = "epoch",
    save_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=5,
    weight_decay=0.01,
    load_best_model_at_end=True,
    metric_for_best_model=metric_name,
)

In [18]:
# source: https://jesusleal.io/2021/04/21/Longformer-multilabel-classification/
def multi_label_metrics(predictions, labels, threshold=0.5):
    # first, apply sigmoid on predictions which are of shape (batch_size, num_labels)
    sigmoid = torch.nn.Sigmoid()
    probs = sigmoid(torch.Tensor(predictions))
    # next, use threshold to turn them into integer predictions
    y_pred = np.zeros(probs.shape)
    y_pred[np.where(probs >= threshold)] = 1
    # finally, compute metrics
    y_true = labels
    f1_micro_average = f1_score(y_true=y_true, y_pred=y_pred, average='micro')
    roc_auc = roc_auc_score(y_true, y_pred, average = 'micro')
    accuracy = accuracy_score(y_true, y_pred)
    # return as dictionary
    metrics = {'f1': f1_micro_average,
               'roc_auc': roc_auc,
               'accuracy': accuracy}
    return metrics

def compute_metrics(p: EvalPrediction):
    preds = p.predictions[0] if isinstance(p.predictions, 
            tuple) else p.predictions
    result = multi_label_metrics(
        predictions=preds, 
        labels=p.label_ids)
    return result

In [19]:
encoded_dataset['train'][0]['labels'].type()

'torch.FloatTensor'

In [20]:
encoded_dataset['train']['input_ids'][0]

tensor([  101,  2197,  7172,  1024,  2089,  2423,  1010, 16798,  2475,  2023,
         9733, 24134,  2326,  2504,  3820,  1006,  2023,  1523, 22889,  2050,
         1524,  1007,  2003,  1037,  3343,  8677,  1996,  2224,  1997,  9733,
        21274, 24134,  6112,  1006,  1523,  9733, 14925,  2475,  1524,  1007,
         1008,  1998, 12033, 10329,  2000,  2169,  4070,  2478,  9733, 14925,
         2475,  1012,   102,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0])

In [21]:
#forward pass
outputs = model(input_ids=encoded_dataset['train']['input_ids'][0].unsqueeze(0), labels=encoded_dataset['train'][0]['labels'].unsqueeze(0))
outputs

SequenceClassifierOutput(loss=tensor(0.7557, grad_fn=<BinaryCrossEntropyWithLogitsBackward0>), logits=tensor([[-0.3193,  0.5522,  0.1132, -0.0092, -0.7527,  0.3493,  0.0312]],
       grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)

In [22]:
trainer = Trainer(
    model,
    args,
    train_dataset=encoded_dataset["train"],
    eval_dataset=encoded_dataset["validation"],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

In [23]:
trainer.train()

C:\Users\elena\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\transformers\optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 58
  Num Epochs = 5
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 40
  Number of trainable parameters = 109487623


  0%|          | 0/40 [00:00<?, ?it/s]

You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
***** Running Evaluation *****
  Num examples = 30
  Batch size = 8


  0%|          | 0/4 [00:00<?, ?it/s]

Saving model checkpoint to bert-finetuned-sem_eval-english\checkpoint-8
Configuration saved in bert-finetuned-sem_eval-english\checkpoint-8\config.json


...

Model weights saved in bert-finetuned-sem_eval-english\checkpoint-16\pytorch_model.bin
tokenizer config file saved in bert-finetuned-sem_eval-english\checkpoint-16\tokenizer_config.json
Special tokens file saved in bert-finetuned-sem_eval-english\checkpoint-16\special_tokens_map.json
***** Running Evaluation *****
  Num examples = 30
  Batch size = 8


  0%|          | 0/4 [00:00<?, ?it/s]

Saving model checkpoint to bert-finetuned-sem_eval-english\checkpoint-24
Configuration saved in bert-finetuned-sem_eval-english\checkpoint-24\config.json


{'eval_loss': 0.5207775235176086, 'eval_f1': 0.2745098039215686, 'eval_roc_auc': 0.5795454545454546, 'eval_accuracy': 0.13333333333333333, 'eval_runtime': 32.4553, 'eval_samples_per_second': 0.924, 'eval_steps_per_second': 0.123, 'epoch': 3.0}


Model weights saved in bert-finetuned-sem_eval-english\checkpoint-24\pytorch_model.bin
tokenizer config file saved in bert-finetuned-sem_eval-english\checkpoint-24\tokenizer_config.json
Special tokens file saved in bert-finetuned-sem_eval-english\checkpoint-24\special_tokens_map.json


Evaluate

After training, we evaluate our model on the validation set.

In [ ]:
trainer.evaluate()

Inference

In [ ]:
text = "With respect to a Cloud Service listed above for which the Availability Service Level Agreement under this subsection applies, Oracle will use commercially reasonable efforts to have each such Service available with a Monthly Uptime Percentage (as defined below) of at least 99.9% during any calendar month (the “Service Commitment”)."

encoding = tokenizer(text, return_tensors="pt")
encoding = {k: v.to(trainer.model.device) for k,v in encoding.items()}

outputs = trainer.model(**encoding)

: 

In [ ]:
logits = outputs.logits
logits.shape

: 

In [ ]:
# apply sigmoid + threshold
sigmoid = torch.nn.Sigmoid()
probs = sigmoid(logits.squeeze().cpu())
predictions = np.zeros(probs.shape)
print(probs)
predictions[np.where(probs >= 0.4)] = 1
# turn predicted id's into actual label names
predicted_labels = [id2label[idx] for idx, label in enumerate(predictions) if label == 1.0]
print(predicted_labels)

: 